In [ ]:
import cv2
import os
from keras.models import load_model
import numpy as np
from pygame import mixer
import time
import tensorflow as tf
from pathlib import Path

In [ ]:
mixer.init()
sound = mixer.Sound('alarm.wav')

face = cv2.CascadeClassifier('haar cascade files\haarcascade_frontalface_alt.xml')
leye = cv2.CascadeClassifier('haar cascade files\haarcascade_lefteye_2splits.xml')
reye = cv2.CascadeClassifier('haar cascade files\haarcascade_righteye_2splits.xml')

In [ ]:
lbl=['Close','Open']

model = load_model('models/cnncat2.h5')
path = os.getcwd()
cap = cv2.VideoCapture(0)  #Comment this for Pre Recorded Access
#cap = cv2.VideoCapture('Videos/PXL_20220309_084625876.mp4') #Comment this for Camera Access
#cap.set(cv2.CAP_PROP_BUFFERSIZE, 5) #Comment this for Camera Access
#cap.set(cv2.CAP_PROP_FPS, 60) #Comment this for Camera Access
font = cv2.FONT_HERSHEY_COMPLEX_SMALL
count=0
score=0
prevScore=0
thicc=2
rpred=[99]
lpred=[99]

In [ ]:
while(True):
    ret, frame = cap.read()
    #cv2.normalize(frame, frame, 0, 255, cv2.NORM_MINMAX)
    #cv2.imshow('frame',frame)
    #if cv2.waitKey(1) & 0xFF == ord('q'):
    #    break
    height,width = frame.shape[:2] 

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #contrast = 1.25
    #brightness = 30
    #frame[:,:,2] = np.clip(contrast * frame[:,:,2] + brightness, 0, 255)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face.detectMultiScale(gray,minNeighbors=5,scaleFactor=1.1,minSize=(25,25))
    left_eye = leye.detectMultiScale(gray)
    right_eye =  reye.detectMultiScale(gray)

    cv2.rectangle(frame, (0,height-50) , (200,height) , (0,0,0) , thickness=cv2.FILLED )

    for (x,y,w,h) in faces:
        cv2.rectangle(frame, (x,y) , (x+w,y+h) , (100,100,100) , 1 )

    for (x,y,w,h) in right_eye:
        r_eye=frame[y:y+h,x:x+w]
        count=count+1
        r_eye = cv2.cvtColor(r_eye,cv2.COLOR_BGR2GRAY)
        r_eye = cv2.resize(r_eye,(24,24))
        r_eye= r_eye/255
        r_eye=  r_eye.reshape(24,24,-1)
        r_eye = np.expand_dims(r_eye,axis=0)
        rpred = (model.predict(r_eye)>0.5).astype('int32')
        if(rpred[0][1]==1).all():
            lbl='Open' 
        if(rpred[0][1]==0).all():
            lbl='Closed'
        break

    for (x,y,w,h) in left_eye:
        l_eye=frame[y:y+h,x:x+w]
        count=count+1
        l_eye = cv2.cvtColor(l_eye,cv2.COLOR_BGR2GRAY)  
        l_eye = cv2.resize(l_eye,(24,24))
        l_eye= l_eye/255
        l_eye=l_eye.reshape(24,24,-1)
        l_eye = np.expand_dims(l_eye,axis=0)
        lpred = (model.predict(l_eye)>0.5).astype('int32')
        if(lpred[0][1]==1).all():
            lbl='Open'   
        if(lpred[0][1]==0).all():
            lbl='Closed'
        break

    if(((rpred[0][1]==0).all() and (lpred[0][1]==0).all())):
        prevScore=score
        score=score+1
        cv2.putText(frame,"Closed",(10,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
    else:
        prevScore=score
        score=score-1
        cv2.putText(frame,"Open",(10,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
        
    if(score<0):
        score=0   
    cv2.putText(frame,'Score:'+str(score),(100,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
    if(score>15):
        #person is feeling sleepy so we beep the alarm
        cv2.imwrite(os.path.join(path,'image.jpg'),frame)
        try:
            sound.play()
            
        except:  # isplaying = False
            pass
        if(thicc<16):
            thicc= thicc+2
        else:
            thicc=thicc-2
            if(thicc<2):
                thicc=2
        cv2.rectangle(frame,(0,0),(width,height),(0,0,255),thicc) 

    if(prevScore > score):
        try:
            sound.stop()
        except:
            print('not playing')    
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [ ]:
#export_dir="saved_models/test_saved_model"
#input_data=tf.constant(1.,shape=[1,1])
#to_save = root.f.get_concrete_function(input_data)
#tf.saved_model.save(root, export_dir, to_save)

#converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
#tflite_model = converter.convert()

model.save('saved_models/model_keras.h5')
model_keras=tf.keras.models.load_model('saved_models/model_keras.h5')

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_keras)
tflite_model = converter.convert()

In [ ]:
import pathlib

tflite_model_file = pathlib.Path('saved_models/pretrainedmodel.tflite')
tflite_model_file.write_bytes(tflite_model)